In [1]:
import tensorflow as tf
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import random

TRAINING_FILE = 'aug_training.txt'
VALIDATION_FILE = 'aug_validation.txt'



learning_rate= 0.001
image_size= 96
image_color= 3
batch_size=100
num_classes=3
maxpool_filter_size= 2
log_dir='C:\\Users\\td170\\' 

#conv_1
conv1_filter_size = 3
conv1_layer_size = 16
stride1 = 1

#conv_2
conv2_filter_size = 3
conv2_layer_size = 32
stride2 = 1

#conv_3
conv3_filter_size = 3
conv3_layer_size = 64
stride3 = 1

#conv_4
conv4_filter_size = 5
conv4_layer_size = 128
stride4 = 1

#fc_1
input_layer_size = 6*6*conv4_layer_size
fc1_layer_size = 256

#fc_2
fc2_layer_size = 256

W_fo = tf.Variable(tf.truncated_normal([fc2_layer_size,num_classes],stddev=0.1))
W_fc2 = tf.Variable(tf.truncated_normal([fc1_layer_size,fc2_layer_size],stddev=0.1))
W_fc1 = tf.Variable(tf.truncated_normal([input_layer_size,fc1_layer_size],stddev=0.1))
l2reg = 0.01 * tf.reduce_sum(tf.square(W_fo))

def get_input_queue(csv_file_name,num_epochs = None):
    train_images = []
    train_labels = []
    for line in open(csv_file_name,'r'):
        cols = re.split(',|\n',line)
        train_images.append(cols[0])
        
        # 3rd column is label and needs to be converted to int type
        train_labels.append(int(cols[2]))
        
                            
    input_queue = tf.train.slice_input_producer([train_images,train_labels],
                                               num_epochs = num_epochs,shuffle = True)
    
    return input_queue

def read_data(input_queue):
    image_file = input_queue[0]
    label = input_queue[1]
    
    image =  tf.image.decode_png(tf.read_file(image_file),channels=image_color)
    
    return image,label,image_file

def read_data_batch(csv_file_name,batch_size=batch_size):
    input_queue = get_input_queue(csv_file_name)
    image,label,file_name= read_data(input_queue)
    print(label)
    image = tf.reshape(image,[image_size,image_size,image_color])
    
   # print(type(image2))
    
    # random image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image,max_delta=0.5)
    image = tf.image.random_contrast(image,lower=0.2,upper=2.0)
    image = tf.image.random_hue(image,max_delta=0.08)
    image = tf.image.random_saturation(image,lower=0.2,upper=2.0)

    
    batch_image,batch_label,batch_file = tf.train.batch([image,label,file_name],batch_size=batch_size)
    #,enqueue_many=True)
    batch_file = tf.reshape(batch_file,[batch_size,1])

    batch_label_on_hot=tf.one_hot(tf.to_int64(batch_label),
        num_classes, on_value=1.0, off_value=0.0)
    print(batch_label_on_hot)
    return batch_image,batch_label_on_hot,batch_file

# convolutional network layer 1
def conv1(input_data):
    # layer 1 (convolutional layer)
    conv1_filter_size = 3
    conv1_layer_size = 16
    stride1 = 1
    
    with tf.name_scope('conv_1'):
        W_conv1 = tf.Variable(tf.truncated_normal(
                        [conv1_filter_size,conv1_filter_size,image_color,conv1_layer_size],
                                              stddev=0.1))
        b1 = tf.Variable(tf.truncated_normal(
                        [conv1_layer_size],stddev=0.1))
        h_conv1 = tf.nn.conv2d(input_data,W_conv1,strides=[1,1,1,1],padding='SAME')
        h_conv1_relu = tf.nn.relu(tf.add(h_conv1,b1))
        h_conv1_maxpool = tf.nn.max_pool(h_conv1_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv1_maxpool

# convolutional network layer 2
def conv2(input_data):
    conv2_filter_size = 3
    conv2_layer_size = 32
    stride2 = 1
    
    with tf.name_scope('conv_2'):
        W_conv2 = tf.Variable(tf.truncated_normal(
                        [conv2_filter_size,conv2_filter_size,conv1_layer_size,conv2_layer_size],
                                              stddev=0.1))
        b2 = tf.Variable(tf.truncated_normal(
                        [conv2_layer_size],stddev=0.1))
        h_conv2 = tf.nn.conv2d(input_data,W_conv2,strides=[1,1,1,1],padding='SAME')
        h_conv2_relu = tf.nn.relu(tf.add(h_conv2,b2))
        h_conv2_maxpool = tf.nn.max_pool(h_conv2_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv2_maxpool

# convolutional network layer 3
def conv3(input_data):
    conv3_filter_size = 3
    conv3_layer_size = 64
    stride3 = 1
    
    print ('## FLAGS.stride1 ',stride1)
    with tf.name_scope('conv_3'):
        W_conv3 = tf.Variable(tf.truncated_normal(
                        [conv3_filter_size,conv3_filter_size,conv2_layer_size,conv3_layer_size],
                                              stddev=0.1))
        b3 = tf.Variable(tf.truncated_normal(
                        [conv3_layer_size],stddev=0.1))
        h_conv3 = tf.nn.conv2d(input_data,W_conv3,strides=[1,1,1,1],padding='SAME')
        h_conv3_relu = tf.nn.relu(tf.add(h_conv3,b3))
        h_conv3_maxpool = tf.nn.max_pool(h_conv3_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv3_maxpool

# convolutional network layer 3
def conv4(input_data):
    conv4_filter_size = 5
    conv4_layer_size = 128
    stride4 = 1
    
    with tf.name_scope('conv_4'):
        W_conv4 = tf.Variable(tf.truncated_normal(
                        [conv4_filter_size,conv4_filter_size,conv3_layer_size,conv4_layer_size],
                                              stddev=0.1))
        b4 = tf.Variable(tf.truncated_normal(
                        [conv4_layer_size],stddev=0.1))
        h_conv4 = tf.nn.conv2d(input_data,W_conv4,strides=[1,1,1,1],padding='SAME')
        h_conv4_relu = tf.nn.relu(tf.add(h_conv4,b4))
        h_conv4_maxpool = tf.nn.max_pool(h_conv4_relu
                                        ,ksize=[1,2,2,1]
                                        ,strides=[1,2,2,1],padding='SAME')
        
        
    return h_conv4_maxpool

# fully connected layer 1
def fc1(input_data):
    input_layer_size = 6*6*conv4_layer_size
    fc1_layer_size = 256
    
    with tf.name_scope('fc_1'):
        # 앞에서 입력받은 다차원 텐서를 fcc에 넣기 위해서 1차원으로 피는 작업
        input_data_reshape = tf.reshape(input_data, [-1, input_layer_size])
        W_fc1 = tf.Variable(tf.truncated_normal([input_layer_size,fc1_layer_size],stddev=0.1))
        b_fc1 = tf.Variable(tf.truncated_normal(
                        [fc1_layer_size],stddev=0.1))
        h_fc1 = tf.add(tf.matmul(input_data_reshape,W_fc1) , b_fc1) # h_fc1 = input_data*W_fc1 + b_fc1
        h_fc1_relu = tf.nn.relu(h_fc1)
    
    return h_fc1_relu
    
# fully connected layer 2
def fc2(input_data):
    fc2_layer_size = 256
    
    with tf.name_scope('fc_2'):
        W_fc2 = tf.Variable(tf.truncated_normal([fc1_layer_size,fc2_layer_size],stddev=0.1))
        b_fc2 = tf.Variable(tf.truncated_normal(
                        [fc2_layer_size],stddev=0.1))
        h_fc2 = tf.add(tf.matmul(input_data,W_fc2) , b_fc2) # h_fc1 = input_data*W_fc1 + b_fc1
        h_fc2_relu = tf.nn.relu(h_fc2)
    
    return h_fc2_relu

# final layer
def final_out(input_data):

    with tf.name_scope('final_out'):
        W_fo = tf.Variable(tf.truncated_normal([fc2_layer_size,num_classes],stddev=0.1))
        b_fo = tf.Variable(tf.truncated_normal(
                        [num_classes],stddev=0.1))
        h_fo = tf.add(tf.matmul(input_data,W_fo) , b_fo) # h_fc1 = input_data*W_fc1 + b_fc1
        
    # 최종 레이어에 softmax 함수는 적용하지 않았다. 
        
    return h_fo

# build cnn_graph
def build_model(images,keep_prob):
    # define CNN network graph
    # output shape will be (*,48,48,16)
    r_cnn1 = conv1(images) # convolutional layer 1
    print ("shape after cnn1 ",r_cnn1.get_shape())
    
    # output shape will be (*,24,24,32)
    r_cnn2 = conv2(r_cnn1) # convolutional layer 2
    print ("shape after cnn2 :",r_cnn2.get_shape() )
    
    # output shape will be (*,12,12,64)
    r_cnn3 = conv3(r_cnn2) # convolutional layer 3
    print ("shape after cnn3 :",r_cnn3.get_shape() )

    # output shape will be (*,6,6,128)
    r_cnn4 = conv4(r_cnn3) # convolutional layer 4
    print ("shape after cnn4 :",r_cnn4.get_shape() )
    
    # fully connected layer 1
    r_fc1 = fc1(r_cnn4)
    print ("shape after fc1 :",r_fc1.get_shape() )

    # fully connected layer2
    r_fc2 = fc2(r_fc1)
    print ("shape after fc2 :",r_fc2.get_shape() )
    
    ## drop out
    # 참고 http://stackoverflow.com/questions/34597316/why-input-is-scaled-in-tf-nn-dropout-in-tensorflow
    # 트레이닝시에는 keep_prob < 1.0 , Test 시에는 1.0으로 한다. 
    r_dropout = tf.nn.dropout(r_fc2,keep_prob)
    print ("shape after dropout :",r_dropout.get_shape() ) 
    
    # final layer
    r_out = final_out(r_dropout)
    print ("shape after final layer :",r_out.get_shape() )


    return r_out 

def main(argv=None):
    
    # define placeholders for image data & label for traning dataset
    
    images = tf.placeholder(tf.float32,[None,image_size,image_size,image_color])
    labels = tf.placeholder(tf.int32,[None,num_classes])
    image_batch,label_batch,file_batch = read_data_batch(TRAINING_FILE) 
    
    keep_prob = tf.placeholder(tf.float32) # dropout ratio
    prediction = build_model(images,keep_prob)
    # define loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=labels))

    tf.summary.scalar('loss',loss)

    #define optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)


    # for validation
    #with tf.name_scope("prediction"):
 
    validate_image_batch,validate_label_batch,validate_file_batch = read_data_batch(VALIDATION_FILE)
    label_max = tf.argmax(labels,1) #라벨이뭔지  
    pre_max = tf.argmax(prediction,1)#예측한라벨이뭔지
    #label_max=labels
    #pre_max=prediction
    correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
            
    tf.summary.scalar('accuracy',accuracy)
        
    
    startTime = datetime.now()
    
    #build the summary tensor based on the tF collection of Summaries
    summary = tf.summary.merge_all()
    
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        saver = tf.train.Saver() # create saver to store training model into file
        summary_writer = tf.summary.FileWriter('./log/',sess.graph)
        
        init_op = tf.global_variables_initializer() # use this for tensorflow 0.12rc0
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        sess.run(init_op)
        
        for i in range(10000):
            images_,labels_ = sess.run([image_batch,label_batch])
            #sess.run(train_step,feed_dict={images:images_,labels:labels_,keep_prob:0.8})
            sess.run(train,feed_dict={images:images_,labels:labels_,keep_prob:0.5})
            
            if i % 10 == 0:
                now = datetime.now()-startTime
                print('## time:',now,' steps:',i)         
                
                # print out training status
                rt = sess.run([label_max,pre_max,loss,accuracy],feed_dict={images:images_ 
                                                          , labels:labels_
                                                          , keep_prob:1.0})
                print ('Prediction loss:',rt[2],' accuracy:',rt[3])
                # validation steps
                validate_images_,validate_labels_ = sess.run([validate_image_batch,validate_label_batch])
                rv = sess.run([label_max,pre_max,loss,accuracy],feed_dict={images:validate_images_ 
                                                          , labels:validate_labels_
                                                          , keep_prob:1.0})
                print ('Validation loss:',rv[2],' accuracy:',rv[3])
                
                #print ('label:',rt[0])
                #print ('predict:',rt[1])
                
            
                if(rv[3] > 0.9):
                    break
                # validation accuracy
                #summary_str = sess.run(summary,feed_dict={images:validate_images_ 
                                                         # , labels:validate_labels_
                                                         # , keep_prob:1.0})
                #summary_writer.add_summary(summary_str,i)
                #summary_writer.flush()
        
        saver.save(sess, 'face_recognition/face_recog') # save session
        coord.request_stop()
        coord.join(threads)
        print('finish')

        r = random.randint(0,25)
        print("Label: ", sess.run(tf.argmax(validate_labels_[r:r + 1], 1)))
        print("Prediction: ", sess.run(tf.argmax(prediction, 1), feed_dict={images: validate_images_[r:r + 1], keep_prob: 1}))
        
       
    
main()

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

## time: 0:19:38.992220  steps: 310
Prediction loss: 1.0167022  accuracy: 0.44
Validation loss: 1.2320362  accuracy: 0.48
## time: 0:20:20.782784  steps: 320
Prediction loss: 1.0469394  accuracy: 0.49
Validation loss: 1.1656209  accuracy: 0.33
## time: 0:21:01.346602  steps: 330
Prediction loss: 0.80933356  accuracy: 0.64
Validation loss: 1.5345747  accuracy: 0.42
## time: 0:21:42.556238  steps: 340
Prediction loss: 0.9257712  accuracy: 0.6
Validation loss: 1.1019075  accuracy: 0.48
## time: 0:22:20.534436  steps: 350
Prediction loss: 0.8547989  accuracy: 0.54
Validation loss: 1.1121799  accuracy: 0.48
## time: 0:22:57.954039  steps: 360
Prediction loss: 0.77289873  accuracy: 0.64
Validation loss: 1.4038945  accuracy: 0.45
## time: 0:23:37.881331  steps: 370
Prediction loss: 0.80597997  accuracy: 0.57
Validation loss: 1.171228  accuracy: 0.48
## time: 0:24:16.903248  steps: 380
Prediction loss: 0.93739563  accuracy: 0.51
Validation loss: 1.2590674  accuracy: 0.38
## time: 0:24:54.44912

## time: 1:01:04.003330  steps: 980
Prediction loss: 0.26250482  accuracy: 0.89
Validation loss: 0.6603882  accuracy: 0.71
## time: 1:01:39.445730  steps: 990
Prediction loss: 0.45101276  accuracy: 0.82
Validation loss: 0.81563276  accuracy: 0.66
## time: 1:02:14.244339  steps: 1000
Prediction loss: 0.42567024  accuracy: 0.84
Validation loss: 0.643  accuracy: 0.7
## time: 1:02:56.905978  steps: 1010
Prediction loss: 0.28100303  accuracy: 0.89
Validation loss: 0.91458094  accuracy: 0.69
## time: 1:03:33.304904  steps: 1020
Prediction loss: 0.3722229  accuracy: 0.85
Validation loss: 1.0835798  accuracy: 0.62
## time: 1:04:16.394102  steps: 1030
Prediction loss: 0.40359116  accuracy: 0.84
Validation loss: 0.90846145  accuracy: 0.65
## time: 1:04:57.507998  steps: 1040
Prediction loss: 0.26695135  accuracy: 0.87
Validation loss: 0.7810991  accuracy: 0.71
## time: 1:05:33.670636  steps: 1050
Prediction loss: 0.29070994  accuracy: 0.9
Validation loss: 0.48459798  accuracy: 0.85
## time: 1:06

Prediction loss: 0.12818658  accuracy: 0.95
Validation loss: 0.77390075  accuracy: 0.81
## time: 1:54:10.712338  steps: 1650
Prediction loss: 0.17348501  accuracy: 0.94
Validation loss: 0.7763539  accuracy: 0.77
## time: 1:55:02.880999  steps: 1660
Prediction loss: 0.11424957  accuracy: 0.94
Validation loss: 1.144851  accuracy: 0.73
## time: 1:55:51.848519  steps: 1670
Prediction loss: 0.07016289  accuracy: 0.96
Validation loss: 0.9709457  accuracy: 0.76
## time: 1:57:00.034048  steps: 1680
Prediction loss: 0.11063137  accuracy: 0.95
Validation loss: 0.7397621  accuracy: 0.81
## time: 1:57:44.621645  steps: 1690
Prediction loss: 0.16071026  accuracy: 0.93
Validation loss: 0.8464291  accuracy: 0.77
## time: 1:58:28.462116  steps: 1700
Prediction loss: 0.13220617  accuracy: 0.95
Validation loss: 0.95743376  accuracy: 0.75
## time: 1:59:12.158037  steps: 1710
Prediction loss: 0.18172692  accuracy: 0.9
Validation loss: 0.96376663  accuracy: 0.82
## time: 1:59:49.081313  steps: 1720
Predict

Prediction loss: 0.086171284  accuracy: 0.96
Validation loss: 1.5487833  accuracy: 0.8
## time: 2:40:00.720605  steps: 2310
Prediction loss: 0.03862706  accuracy: 0.98
Validation loss: 1.0818559  accuracy: 0.8
## time: 2:40:47.235986  steps: 2320
Prediction loss: 0.053555746  accuracy: 0.98
Validation loss: 0.5888156  accuracy: 0.77
## time: 2:41:44.703190  steps: 2330
Prediction loss: 0.031120576  accuracy: 0.99
Validation loss: 1.1242027  accuracy: 0.85
## time: 2:42:29.519878  steps: 2340
Prediction loss: 0.08097595  accuracy: 0.96
Validation loss: 0.8141188  accuracy: 0.83
## time: 2:43:25.427176  steps: 2350
Prediction loss: 0.161176  accuracy: 0.93
Validation loss: 1.2835006  accuracy: 0.8
## time: 2:44:24.316337  steps: 2360
Prediction loss: 0.12465279  accuracy: 0.94
Validation loss: 1.1756655  accuracy: 0.77
## time: 2:45:15.037000  steps: 2370
Prediction loss: 0.046110444  accuracy: 0.99
Validation loss: 0.5806823  accuracy: 0.8
## time: 2:45:58.399776  steps: 2380
Prediction

Validation loss: 1.2835248  accuracy: 0.79
## time: 3:31:21.104148  steps: 2970
Prediction loss: 0.05959048  accuracy: 0.98
Validation loss: 1.2517561  accuracy: 0.8
## time: 3:32:03.943561  steps: 2980
Prediction loss: 0.051419884  accuracy: 0.99
Validation loss: 1.5525994  accuracy: 0.8
## time: 3:32:47.075408  steps: 2990
Prediction loss: 0.037923697  accuracy: 0.99
Validation loss: 0.6912637  accuracy: 0.85
## time: 3:33:34.641943  steps: 3000
Prediction loss: 0.04225998  accuracy: 0.99
Validation loss: 0.65956  accuracy: 0.83
## time: 3:34:27.090251  steps: 3010
Prediction loss: 0.04159089  accuracy: 0.98
Validation loss: 1.1766723  accuracy: 0.74
## time: 3:35:22.292178  steps: 3020
Prediction loss: 0.06726622  accuracy: 0.97
Validation loss: 0.7337831  accuracy: 0.8
## time: 3:36:08.121737  steps: 3030
Prediction loss: 0.09161354  accuracy: 0.97
Validation loss: 1.216286  accuracy: 0.7
## time: 3:36:51.025225  steps: 3040
Prediction loss: 0.055021185  accuracy: 0.99
Validation l